In [1]:
# ray tunning experiment-II  , for best of 1000 samples 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune import PlacementGroupFactory
from ray.air import Checkpoint, session
import ray

# Load data (replace '/path/to/spin_Ai_data.csv' with your data path)
final_df= pd.read_csv("/home/rs/21CS91R11/vikash/final_formula_df.csv" , header = None)
Y = final_df[0].values
X = final_df.drop(0,axis=1).values

# Preprocess data
sc = StandardScaler()
X = sc.fit_transform(X)
x_train, x_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)


# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.float)
        self.targets = torch.tensor(targets, dtype=torch.float)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_item = self.inputs[idx]
        target_item = self.targets[idx]
        return input_item, target_item


def extract_data():
    train_dataset = CustomDataset(x_train, y_train)
    val_dataset = CustomDataset(x_val, y_val)
    test_dataset = CustomDataset(x_test, y_test)
    return train_dataset, val_dataset, test_dataset


# ANN Model
class ANNModel(nn.Module):
    def __init__(self, input_features=112, hidden1=56, hidden2=28, out_features=1):
        super().__init__()
        self.fc1 = nn.Linear(input_features, hidden1)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.dropout2 = nn.Dropout(p=0.5)
        self.out = nn.Linear(hidden2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.out(x)
        return torch.sigmoid(x)


# Train and Validation Function
def train_and_val(config, checkpoint_dir=None):
    model = ANNModel(input_features=112, hidden1=config['h1'], hidden2=config['h2'], out_features=1)
    train_dataset, val_dataset, _ = extract_data()

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    torch.manual_seed(20)
    loss_function = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

    checkpoint = session.get_checkpoint()
    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        start_epoch = checkpoint_state["epoch"]
        model.load_state_dict(checkpoint_state["model_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=int(config["batch_size"]),
                                                  shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=int(config["batch_size"]),
                                                shuffle=True)

    for epoch in range(start_epoch,600):
        train_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(train_dataloader, 0):
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(f"[{epoch + 1}, {i + 1}] loss: {train_loss / epoch_steps:.3f}")
                train_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        with torch.no_grad():
            for i, data in enumerate(val_dataloader, 0):
                inputs, targets = data
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                predicted = (outputs>=0.5).float()
                total += targets.size(0)
               # targets = targets.round()
                correct += (predicted.squeeze() == targets).sum().item()

                loss = loss_function(outputs, targets)
                val_loss += loss.item()
                val_steps += 1

        tune.report(loss=val_loss / val_steps, accuracy=correct / total)
        


def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    ray.init()

    config = {
        "h1": tune.choice([int(x) for x in np.linspace(1, 112, 112)]),
        "h2": tune.choice([int(x) for x in np.linspace(1, 112, 112)]),
        "lr": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([int(x) for x in np.linspace(500,5000, 1000)]),
        "weight_decay": tune.choice([0.01, 0.0001, 0.001])
    }

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2
    )

    result = tune.run(
        train_and_val,
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
       # resources_per_trial=PlacementGroupFactory([{'CPU': 40.0}] + [{'CPU': 1.0}] * 2)
        resources_per_trial=PlacementGroupFactory([{"CPU": 48.0}, {"GPU": 1.0}])
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = ANNModel(input_features=112, hidden1=best_trial.config["h1"],
                                  hidden2=best_trial.config["h2"], out_features=1)
    
    
       


2023-08-06 15:02:23,264	INFO util.py:125 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-06 15:02:23,776	INFO util.py:125 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=1000, max_num_epochs=600, gpus_per_trial=1)

2023-08-06 15:02:28,454	INFO worker.py:1621 -- Started a local Ray instance.
2023-08-06 15:02:30,642	INFO tune.py:658 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-08-06 15:02:30,644	WARNING syncer.py:261 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consider installing the respective fsspec-package (see https://github.com/fsspec).
/home/rs/21CS91R11/anaconda3/envs/envpytorch3/lib/python3.7/site-packages/ray/tune/trainable/function_trainable.py:626: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import sess

2023-08-06 15:02:37,426	WARNING worker.py:2006 -- Warning: The actor ImplicitFunc is very large (17 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(train_and_val pid=40958) /home/rs/21CS91R11/anaconda3/envs/envpytorch3/lib/python3.7/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([2247])) that is different to the input size (torch.Size([2247, 1])) is deprecated. Please ensure they have the same size.
(train_and_val pid=40958)   return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
(train_and_val pid=40958) /home/rs/21CS91R11/anaconda3/envs/envpytorch3/lib/python3.7/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([1486])) that is different to the input size (torch.Size([1486, 1])) is deprecated. Please ensure they have the same size.
(train_and_val pid=4

Trial name,accuracy,loss
train_and_val_2a49e_00000,0.863946,0.344493
train_and_val_2a49e_00001,0.735666,0.622984
train_and_val_2a49e_00002,0.655005,0.664422
train_and_val_2a49e_00003,0.562682,0.676983
train_and_val_2a49e_00004,0.736638,0.525767
train_and_val_2a49e_00005,0.634597,0.670198
train_and_val_2a49e_00006,0.600583,0.676807
train_and_val_2a49e_00007,0.634597,0.59601
train_and_val_2a49e_00008,0.726919,0.645028
train_and_val_2a49e_00009,0.870748,0.318791


(train_and_val pid=41420) /home/rs/21CS91R11/anaconda3/envs/envpytorch3/lib/python3.7/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([3283])) that is different to the input size (torch.Size([3283, 1])) is deprecated. Please ensure they have the same size.
(train_and_val pid=41420)   return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
(train_and_val pid=41420) /home/rs/21CS91R11/anaconda3/envs/envpytorch3/lib/python3.7/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([1661])) that is different to the input size (torch.Size([1661, 1])) is deprecated. Please ensure they have the same size.
(train_and_val pid=41420)   return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
(train_and_val pid=41420) /home/rs/21CS91R11/anaconda3/envs/envpytorch3/lib/python3.7/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (to

Best trial config: {'h1': 85, 'h2': 29, 'lr': 0.002701949763237993, 'batch_size': 729, 'weight_decay': 0.001}
Best trial final validation loss: 0.2971115708351135
Best trial final validation accuracy: 0.8726919339164237
